Install and import

!pip install mne

In [3]:
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import scipy.io
import mne
from mne import create_info
from mne.io import RawArray
from mne.datasets import eegbci
from mne.io import concatenate_raws, read_raw_edf
from numpy import load

Preprocess

In [4]:
y = load('datay.npy')
X = load('dataX.npy')

In [10]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33)

In [11]:
dataset = []
arr = np.arange(len(X))
np.random.shuffle(arr)

for n in range(len(X)):
    dataset.append((X[arr[n]], y[arr[n]]))

In [13]:
split_valid = 0.3
n_valid = int(split_valid*len(X))

dataset_train = dataset[0:len(X)-n_valid]
dataset_valid = dataset[len(X)-n_valid:len(X)]

count_left = 0
count_right = 0
for n in range(len(dataset_train)):
    if dataset_train[n][1] == 2:
        count_left = count_left+1
    else:
        count_right = count_right+1
print('There are {} left and {} right in training set.'.format(count_left, count_right))

count_left = 0
count_right = 0
for n in range(len(dataset_valid)):
    if dataset_train[n][1] == 2:
        count_left = count_left+1
    else:
        count_right = count_right+1
print('There are {} left and {} right in validation set.'.format(count_left, count_right))

There are 31 left and 32 right in training set.
There are 11 left and 16 right in validation set.


In [14]:
batch_size = 1024
train_loader = torch.utils.data.DataLoader(dataset_train, batch_size=batch_size, shuffle=False, num_workers=1)
valid_loader = torch.utils.data.DataLoader(dataset_valid, batch_size=batch_size, shuffle=False, num_workers=1)

In [16]:
import numpy as np
from sklearn.metrics import roc_auc_score, precision_score, recall_score, accuracy_score
import torch
import torch.nn as nn
import torch.cuda as cuda
import torch.optim as optim
from torch.autograd import Variable
import torch.nn.functional as F
import torch.optim as optim
import matplotlib.pyplot as plt
from torch.nn import MaxPool1d
class CNN_Model_v1(nn.Module):
    def __init__(self):
        super(CNN_Model_v1, self).__init__()
        # Layer 1
        self.conv1 = nn.Conv1d(in_channels=1,out_channels=25,kernel_size= (11, 1, 1, 25))
        self.conv2 = nn.Conv1d(in_channels=25,out_channels=25,kernel_size= (1, 2, 25, 25))
        self.maxpool1 = MaxPool1d(kernel_size=(1,3,1,1), stride=(1,3,1,1))
        self.conv3 = nn.Conv1d(in_channels=25,out_channels=50,kernel_size= (11, 1, 25, 50))
        self.maxpool2 = MaxPool1d(kernel_size=(1,3,1,1), stride=(1,3,1,1))
        self.conv4 = nn.Conv1d(in_channels=50,out_channels=100,kernel_size= (11, 1, 50, 100))
        self.maxpool3 = MaxPool1d(kernel_size=(1,3,1,1), stride=(1,3,1,1))
        self.conv5 = nn.Conv1d(in_channels=100,out_channels=200,kernel_size= (11, 1, 100, 200))
        self.maxpool4 = MaxPool1d(kernel_size=(1,2,1,1), stride=(1,2,1,1))
        self.fc1 = nn.Linear(800, 2)




    def forward(self, x):
        # Layer 1
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.maxpool1(x)
        x = self.conv3(x)
        x = self.maxpool2(x)
        x = self.conv4(x)
        x = self.maxpool3(x)
        x = self.conv5(x)
        x = self.maxpool4(x)
        x = nn.sigmoid(self.fc1(x))
        return x




In [18]:
# The model
net = CNN_Model_v1()

if cuda.is_available():
    net = net.cuda(3)

# Our loss function
criterion = nn.CrossEntropyLoss()

# Our optimizer
learning_rate = 0.1
optimizer = torch.optim.SGD(net.parameters(), lr=learning_rate, momentum=0.9) 

NameError: name 'cuda' is not defined

In [12]:
num_epochs = 600

train_loss = []
valid_loss = []
train_accuracy = []
valid_accuracy = []

for epoch in range(num_epochs):
    iter_loss = 0.0
    correct = 0
    iterations = 0
    
    net.train()
    
    for i, (items, classes) in enumerate(train_loader):
        items = Variable(items)
        classes = Variable(classes)
        
        if cuda.is_available():
            items = items.cuda(3)
            classes = classes.cuda(3)
        
        optimizer.zero_grad()
        outputs = net(items)
        loss = criterion(outputs, classes)
        iter_loss += loss.item()
        loss.backward()
        optimizer.step()
        
        _, predicted = torch.max(outputs.data, 1)
        correct += (predicted == classes.data).sum()
        iterations += 1

    train_loss.append(iter_loss/iterations)
    
    train_accuracy.append((100 * correct.float() / len(train_loader.dataset)))
    
    loss = 0.0
    correct = 0
    iterations = 0

    net.eval()
    
    for i, (items, classes) in enumerate(valid_loader):
        items = Variable(items)
        classes = Variable(classes)
        
        if cuda.is_available():
            items = items.cuda(3)
            classes = classes.cuda(3)
        
        outputs = net(items)
        loss += criterion(outputs, classes).item()
        
        _, predicted = torch.max(outputs.data, 1)
        correct += (predicted == classes.data).sum()
        
        iterations += 1

    valid_loss.append(loss/iterations)
    correct_scalar = np.array([correct.clone().cpu()])[0]
    valid_accuracy.append(correct_scalar / len(valid_loader.dataset) * 100.0)
    
    print ('Epoch %d/%d, Tr Loss: %.4f, Tr Acc: %.4f, Val Loss: %.4f, Val Acc: %.4f'
               %(epoch+1, num_epochs, train_loss[-1], train_accuracy[-1], valid_loss[-1], valid_accuracy[-1]))

array([3., 2., 2., 3., 3., 2., 3., 2., 3., 2., 2., 3., 2., 3., 2., 2., 3.,
       2., 3., 2., 3., 3., 2., 2., 3., 3., 2., 3., 2., 2., 2., 3., 3., 2.,
       3., 2., 2., 3., 3., 2., 2., 3., 3., 2., 3., 3., 2., 2., 3., 3., 2.,
       2., 3., 2., 3., 3., 2., 2., 3., 2., 2., 3., 2., 3., 2., 3., 2., 3.,
       3., 2., 2., 3., 3., 2., 2., 3., 2., 3., 2., 2., 3., 3., 2., 2., 3.,
       3., 2., 3., 2., 3.], dtype=float32)